## Local navigation

Thymio needs to avoid obstacles, in this case asteroids, as it travels through space. To do this, we call the function local_avoidance() that allows Thymio to avoid the obstacle and continue it way to Mars.

>### Input:
>>-  Current motors speed, horizotnal proximity sensors values

>This function will take the current value of the motors and, depending on the weights and values of the various sensors, adjust the values of the motors so that it dodges the obstacle.

>### Output:
>>- new motors values (new motor_left_target,motor_right_target) allowing to avoid the obstacle
>>- To avoid the robot to be blocked when it faces a front obstacle we adjust the weight so that it has a higher tendency to go on left.

>### Call:
>>- At each step of global navigation

>In reality, when the global path function is called, it generates a path with segments. It then calls the drive() function, which splits the segment into 10 smaller segments. This is where we call the get_prox() function to compare the value of the proximity sensors with our WALL_THRESHOLD. This method gives the robot better detection on its path between two points on the layout. 

>### Limitations:
>The first limitation of this implementation lies in the call. Dividing the distance between 2 points into smaller distances partially solves the problem, because if an obstacle appears between these two smaller distances, the previous problem reappears. However, we don't do this when we're turning, which may mean that the robot doesn't detect obstacles properly when repositioning itself. 
A better approach would have been to use two different threads in parallel. Because what can happen is that the robot oscillates between the global_path() and local_navigation() functions and jerks. 

>### Keys parameters:

| Name | Meaning | Type (Unit) | Global|
|-----------|-----------|-----------|-----------|
|max_allowed_speed|Limit the speed outpout of the motor in local_navigation|int|no|
|mem_sensor|Store the values of the 5 front sensors get by get_prox()|int|no|
|sensor_scale|Rescale the output value of the sensors|int|no|
|WALL_THRESHOLD|Threshold to determined when there is an obsctacle|int|yes||
|ROBOT_SPEED|Current robot speed|int|yes|

In [2]:
@tdmclient.notebook.sync_var
def get_prox():
    global prox_horizontal
    return prox_horizontal


#local navigation function
def local_navigation():
    global WALL_THRESHOLD,ROBOT_SPEED
    max_allowed_speed = 150
    set_var(leds_top = [32, 0, 0])
    while max(get_prox())>WALL_THRESHOLD:
        weight_left = [25,  15, -20, -15, -25]
        weight_right = [-25, -15, -15,  15,  25]
    
        # Scale factors for sensors and constant factor
        sensor_scale = 500
    
        mem_sensor = [0,0,0,0,0]
        prox_horizontal = get_prox()
        
        for i in range(len(mem_sensor)):
            # Get and scale inputs
            mem_sensor[i] = prox_horizontal[i] // sensor_scale
    
        y = [ROBOT_SPEED,ROBOT_SPEED]   
        
        for i in range(len(mem_sensor)):   
            # Compute outputs of neurons and set motor powers
            y[0] = y[0] + mem_sensor[i] * weight_left[i]
            y[1] = y[1] + mem_sensor[i] * weight_right[i]
    
        # Set motor powers
        set_var(motor_left_target = min(y[0],max_allowed_speed))
        set_var(motor_right_target = min(y[1],max_allowed_speed))
        time.sleep(0.2)
    set_var(leds_top = [0, 0, 32])

    return


NameError: name 'tdmclient' is not defined

## Reference:
[1] Solutions Week 3 - Artificial neural networks.ipynb

[2] Some structure of the jupyter notebooks has been borrowed by the GitHub Repo https://github.com/hibetterheyj/Thymio_Vision_Guided_Navigation/blob/master/BMR_Final_Project.ipynb 

## Introduction:
> The aim of this project was to integrate the different themes currently being seen in one project: path planning, vision, local navigation and filtering. To combine these, we treated our robot as a spaceship.
 
>Thymio Mars V aims to reach Mars in the shortest possible time, avoiding black holes along the way. To do this, the camera detects the robot's position, the position of Mars and the position of the black holes. To determine the shortest route, we decided to use the A* algorithm, as it's easy to implement and doesn't require a lot of resources. If Thymio Mars V encounters an asteroid field on its path, obstacle detection takes control and, thanks to the proximity sensors, the robot is able to avoid it.

>Finally, Markov Filtering is used to be sure that the robot starts at the right position because this type of filtering is effective when there is uncertainty about the position and speed, which is the case in this project.